In [ ]:
# 1. 安裝 Chromium 瀏覽器與驅動
!apt-get update
!apt-get install chromium-chromedriver

# 2. 安裝 Selenium
!pip install selenium

# 3. 安裝中文字型 (解決 PDF 亂碼)
!apt-get install -y fonts-noto-cjk

print("✅ 環境安裝完成！請繼續執行第 2 步。")

In [ ]:
# ================= 使用者設定區 =================

# 1. 設定 Email 帳號密碼
gmail_user = 'sanctimagical@gmail.com'
gmail_password = 'hikyuhxdyohsibcv'
to_email = 'sanctimagical@gmail.com'

# 2. 設定統編清單 (直接寫在這裡！)
# 格式：用引號包起來，中間用逗號隔開。每一行代表一批 (最多10個)。
tax_id_batches = [
    "83203327,22211681,81127176",              # 第 1 批
    "83203327,22211681,81127176,27177418",     # 第 2 批 (含外商範例)
    # "11111111,22222222,33333333",            # 您可以隨時新增第 3 批...
]

# ===============================================
print(f"✅ 設定已載入！")
print(f"📋 共準備執行 {len(tax_id_batches)} 批次查詢")

In [ ]:
import time
import base64
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.header import Header
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# --- 啟動瀏覽器 ---
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1080')
options.add_argument('--remote-debugging-port=9222')

try:
    driver = webdriver.Chrome(options=options)
    print("✅ 瀏覽器啟動成功！準備開始批次作業...")
except:
    options.binary_location = "/usr/bin/google-chrome"
    driver = webdriver.Chrome(options=options)
    print("✅ 瀏覽器啟動成功 (備用模式)！")

# 💡 全域變數：記錄「商業」是否已勾選 (防止重複點擊導致取消)
is_business_checked = False

try:
    # --- 開始大迴圈 (讀取步驟2設定的清單) ---
    for batch_index, query_numbers in enumerate(tax_id_batches):
        # 清理一下資料，避免前後有空白
        query_numbers = query_numbers.strip()
        if not query_numbers: continue

        print(f"\n==========================================")
        print(f"🚀 正在執行第 {batch_index + 1} 批次")
        print(f"📋 統編清單: {query_numbers}")
        print(f"==========================================")

        generated_pdfs = []

        # [動作 A] 搜尋與下載
        try:
            driver.get("https://findbiz.nat.gov.tw/fts/query/QueryBar/queryInit.do")
            wait = WebDriverWait(driver, 20)

            # 1. 切換模式
            try:
                # 點擊多統編按鈕
                btn = wait.until(EC.presence_of_element_located((By.XPATH, "//label[contains(text(),'多統編查詢')]")))
                driver.execute_script("arguments[0].click();", btn)
                time.sleep(1) # 稍微緩衝
            except:
                pass

            # 2. 勾選 [公司] (每次都檢查，沒勾就補勾)
            try:
                cmpy_box = driver.find_element(By.XPATH, "//input[@value='isCmpy']")
                if not cmpy_box.is_selected():
                    driver.execute_script("arguments[0].click();", cmpy_box)
                    print("   -> [v] 已勾選 公司")
            except:
                pass

            # 3. 勾選 [商業] (智慧策略：只在第一次勾選，之後跳過)
            try:
                if not is_business_checked:
                    bus_label = driver.find_element(By.XPATH, "//label[contains(text(),'商業')]")
                    driver.execute_script("arguments[0].click();", bus_label)
                    is_business_checked = True
                    print("   -> [v] 首次勾選：商業 (透過標籤)")
                else:
                    print("   -> [skip] 商業已經勾過了，跳過點擊以免取消。")
            except:
                print("   ⚠️ 警告：無法勾選 [商業]")

            # 4. 輸入統編並搜尋
            input_box = wait.until(EC.presence_of_element_located((By.ID, "qryCond")))
            input_box.clear()
            input_box.send_keys(query_numbers)

            search_btn = driver.find_element(By.ID, "qryBtn")
            driver.execute_script("arguments[0].click();", search_btn)

            print("   等待搜尋結果 (5秒)...")
            time.sleep(5)

            # 5. 定位資料
            target_elements = driver.find_elements(By.XPATH, "//*[contains(text(), '統一編號')]/ancestor::tr//a | //*[contains(text(), '統一編號')]/ancestor::div[contains(@class, 'panel')]//a")
            valid_links_indices = [i for i, elem in enumerate(target_elements) if len(elem.text) > 4]

            # 備用定位
            if not valid_links_indices:
                all_links = driver.find_elements(By.TAG_NAME, "a")
                valid_links_indices = [i for i, elem in enumerate(all_links) if len(elem.text) > 5 and "更多" not in elem.text and "指南" not in elem.text]

            print(f"   -> 本批次找到 {len(valid_links_indices)} 筆資料，開始下載...")

            # 6. 下載迴圈
            count = 0
            for _ in valid_links_indices:
                count += 1
                try:
                    # 重新定位
                    current_targets = driver.find_elements(By.XPATH, "//*[contains(text(), '統一編號')]/ancestor::tr//a | //*[contains(text(), '統一編號')]/ancestor::div[contains(@class, 'panel')]//a")
                    if not current_targets:
                        current_targets = [x for x in driver.find_elements(By.TAG_NAME, "a") if len(x.text) > 5 and "更多" not in x.text]

                    if count <= len(current_targets):
                        target = current_targets[count-1]
                        company_name = target.text

                        driver.execute_script("arguments[0].click();", target)
                        time.sleep(4)

                        filename = f"Batch{batch_index+1}_{company_name}.pdf"
                        filename = "".join([c for c in filename if c.isalpha() or c.isdigit() or c in ' ._-' or '\u4e00' <= c <= '\u9fff']).rstrip()

                        print_options = {
                            'landscape': False, 'displayHeaderFooter': False, 'printBackground': True,
                            'preferCSSPageSize': True, 'paperWidth': 8.27, 'paperHeight': 11.69
                        }
                        result = driver.execute_cdp_cmd("Page.printToPDF", print_options)

                        with open(filename, "wb") as f:
                            f.write(base64.b64decode(result['data']))

                        generated_pdfs.append(filename)
                        print(f"      [v] 已下載: {filename}")

                        driver.back()
                        time.sleep(2)
                except Exception as e:
                    print(f"      [x] 下載失敗: {e}")
                    driver.back()
                    time.sleep(2)

        except Exception as e:
            print(f"   ⚠️ 本批次搜尋發生錯誤: {e}")

        # [動作 B] 寄信
        if generated_pdfs:
            print(f"   📧 正在寄送第 {batch_index + 1} 批次的 Email...")
            msg = MIMEMultipart()
            subject = f'商工登記查詢結果 - 第 {batch_index + 1} 批 (共 {len(generated_pdfs)} 筆)'
            msg['Subject'] = Header(subject, 'utf-8')
            msg['From'] = gmail_user
            msg['To'] = to_email

            body = f"這是自動化程式執行的第 {batch_index + 1} 批次查詢結果。\n查詢統編：{query_numbers}\n\n附件包含：\n" + "\n".join(generated_pdfs)
            msg.attach(MIMEText(body, 'plain', 'utf-8'))

            for pdf_file in generated_pdfs:
                try:
                    with open(pdf_file, "rb") as f:
                        attach = MIMEApplication(f.read(), _subtype="pdf")
                        encoded_filename = Header(pdf_file, 'utf-8').encode()
                        attach.add_header('Content-Disposition', 'attachment', filename=encoded_filename)
                        msg.attach(attach)
                except Exception as e:
                    print(f"      [!] 附件夾帶失敗 ({pdf_file}): {e}")

            try:
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()
                server.login(gmail_user, gmail_password)
                server.send_message(msg)
                server.quit()
                print("   ✅ Email 寄送成功！")
            except Exception as e:
                print(f"   ❌ Email 寄送失敗: {e}")
        else:
            print("   ⚠️ 本批次沒有產生任何 PDF，跳過寄信。")

        print("   (休息 5 秒後執行下一批...)")
        time.sleep(5)

    print("\n🎉🎉🎉 所有批次作業全部完成！ 🎉🎉🎉")

except Exception as e:
    print(f"發生嚴重錯誤: {e}")

finally:
    driver.quit()